In [1]:
# requirements for this example:
%pip install -qq \
    quackling \
    python-dotenv \
    langchain-text-splitters \
    langchain-huggingface \
    langchain-milvus

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

FILE_PATH = "https://arxiv.org/pdf/2206.01062"  # DocLayNet paper

In [3]:
import warnings

warnings.filterwarnings(action="ignore", category=UserWarning, module="pydantic|torch")
warnings.filterwarnings(action="ignore", category=FutureWarning, module="easyocr")

## Setup

### Loader and splitter

In [4]:
from quackling.langchain.loaders import DoclingPDFLoader

Below we set up:
- a `Loader` which will be used to create LangChain documents, and
- a splitter, which will be used to split these documents

#### Using JSON

To leverage Docling's rich document structure format, we namely set the parse type to JSON and use a `HierarchicalJSONSplitter` accordingly:

In [5]:
from quackling.langchain.loaders import DoclingPDFLoader
from quackling.langchain.splitters import HierarchicalJSONSplitter

loader = DoclingPDFLoader(
    file_path=FILE_PATH,
    parse_type=DoclingPDFLoader.ParseType.JSON,
)
text_splitter = HierarchicalJSONSplitter()

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

#### Using Markdown

Alternatively, to just use the flat Markdown export instead of the native document format, one can uncomment and use the following:

In [6]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# loader = DoclingPDFLoader(
#     file_path=FILE_PATH,
#     parse_type=DoclingPDFLoader.ParseType.MARKDOWN,
# )
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200,
# )

We now used the above-defined objects to get the document splits:

In [7]:
docs = loader.load()
splits = text_splitter.split_documents(docs)

### Embeddings

In [8]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

HF_EMBED_MODEL_ID = "BAAI/bge-small-en-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=HF_EMBED_MODEL_ID)

### Vector store

In [9]:
from tempfile import TemporaryDirectory

from langchain_milvus import Milvus

MILVUS_URI = os.environ.get(
    "MILVUS_URL", f"{(tmp_dir := TemporaryDirectory()).name}/milvus_demo.db"
)

vectorstore = Milvus.from_documents(
    splits,
    embeddings,
    connection_args={"uri": MILVUS_URI},
    drop_old=True,
)

### LLM

In [10]:
from langchain_huggingface import HuggingFaceEndpoint

HF_API_KEY = os.environ.get("HF_API_KEY")
HF_LLM_MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"

llm = HuggingFaceEndpoint(
    repo_id=HF_LLM_MODEL_ID,
    huggingfacehub_api_token=HF_API_KEY,
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/pva/.cache/huggingface/token
Login successful


## RAG

In [11]:
from typing import Iterable

from langchain_core.documents import Document as LCDocument
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs: Iterable[LCDocument]):
    return "\n\n".join(doc.page_content for doc in docs)


retriever = vectorstore.as_retriever()

prompt = PromptTemplate.from_template(
    "Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {question}\nAnswer:\n"
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
rag_chain.invoke("How many pages were human annotated for DocLayNet?")

'The number of pages annotated for DocLayNet is 80,863.'